In [3]:
import fiona
import folium
import geopandas
from math import pi, sqrt
import miniball #https://github.com/weddige/miniball
from os import remove
from pyproj import Proj, transform
import shapely
from shapely.geometry import Polygon

# math for scores adapted from https://www.azavea.com/blog/2016/07/11/measuring-district-compactness-postgis/

#variables
inputData = "districtShapes/test.shp"
outputData = "test_out/scored_districts.geojson"

colors = ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494'] #ColorBrewer Yell0w-Green-Blue
colors.reverse()

# functions
def reprojectPolygons(data, output_file, output_type, schema, new_crs):
    '''Transform projection. Outputs new file. Only for polygons.'''
    old_crs_proj = Proj(data.crs)
    new_crs_proj = Proj(new_crs)
    with fiona.open(output_file, "w", output_type, schema, new_crs) as output:
        for feat in data:
            polygon_list = []
            geom = feat['geometry']['coordinates']
            gtype = feat['geometry']['type']
            state = feat['properties']['STATENAME']
            dist = feat['properties']['DISTRICT']
            print(state, dist)
            if gtype == 'Polygon':
                #print("Processing", gtype)
                for part in geom:
                    new_part = []
                    for coords in part:
                        x, y = coords
                        #print("Orig coords: ", x, y)
                        new_x, new_y = transform(old_crs_proj, new_crs_proj, x, y)
                        new_coords = (new_x, new_y)
                        #print("New coords: ", new_x, new_y)
                        new_part.append(new_coords)
                        #print(">>>Coords appended")
                    polygon_list.append(new_part)
            elif gtype == 'MultiPolygon':
                #print("Processing", gtype)
                for parts in geom:
                    new_parts = []
                    for part in parts:
                        new_part = []
                        for coords in part:
                            x, y = coords
                            #print("Orig coords: ", x, y)
                            new_x, new_y = transform(old_crs_proj, new_crs_proj, x, y)
                            new_coords = (new_x, new_y)
                            #print("New coords: ", new_x, new_y)
                            new_part.append(new_coords)
                            #print(">>>Coords appended")
                        new_parts.append(new_part)
                    polygon_list.append(new_parts)
            feat['geometry']['coordinates'] = polygon_list
            #print(">Polygon geom written")
            output.write(feat)
        print(data.path, "Tranformation Complete")

def getClassRanges(inputData, field, colorList):
    '''construct list of ranges and corresponding color to use'''
    num_classes = len(colorList)
    colorList = colorList
    data = []
    classes = []
    ranges = []
    
    for feature in inputData:
        value = feature['properties'][field]
        if value != None:
            data.append(value)
        
    step = (max(data)-min(data))/num_classes
    
    for i in range(num_classes):
        classes.append(max(data)-(i * step))

    for i in range(len(classes)-1):
        ranges.append((classes[i], classes[i+1], colorList[i]))
    ranges.append((classes[-1], 0, colorList[-1]))
    return ranges
        
def getColor(feature, field, range_set):
    '''return fill color based on classification range'''
    val = feature['properties'][field]
    if val != None:
        for i in range(len(range_set)):
            upper = range_set[i][0]
            lower = range_set[i][1]
            color = range_set[i][2]
            if val <= upper and val > lower:
                return {'fillcolor':color}
                break
    else:
        return {'fillcolor':'#d3d3d3'}

# classes            
class CompactnessScores(object):
    fields = {"Polsby-Popper":"float","Schwartzberg":"float", "Convex-Hull":"float", "Reock":"float"}
    albersEA = {'init':"EPSG:5070"}
    def __init__(self, polygons, output):
        self.polygons = polygons
        self.final_output = output
        self.temp_output = "_temp.".join(self.final_output.split("."))
        
        #create output file w/new fields
        self.processGeo()

    def processGeo(self):
        with fiona.open(self.polygons) as in_polygons:
            schema = in_polygons.schema.copy()
            
            #got rid of projection check. Too many different formats for crs to try to compare
            reprojectPolygons(in_polygons, self.temp_output, "GeoJSON", schema, self.albersEA)
            
            #add fields to hold compactness scores
            for key in self.fields:
                schema['properties'][key] = self.fields[key]
            print("Schema Updated")
            
        #calc compactness scores
        with fiona.open(self.temp_output, "r", "GeoJSON", schema) as in_polygons:
            with fiona.open(self.final_output,"w","GeoJSON", schema,in_polygons.crs) as out_polygons:
                for feature in in_polygons:
                    gtype = feature['geometry']['type']
                    properties = feature['properties']
                            
                    if gtype == 'Polygon':
                        properties['Polsby-Popper'] = self.calcPolsbyPopper(feature)
                        properties['Schwartzberg'] = self.calcSchwartzberg(feature)
                        properties['Convex-Hull'] = self.calcConvexHull(feature)
                        properties['Reock'] = self.calcReock(feature)
                        feature['properties'] = properties
                        out_polygons.write(feature)
                    else:
                        properties['Polsby-Popper'] = None
                        properties['Schwartzberg'] = None
                        properties['Convex-Hull'] = None
                        properties['Reock'] = None
                        feature['properties'] = properties
                        out_polygons.write(feature)
        remove(self.temp_output)

        print("Processing Successful")
    
    def calcPolsbyPopper(self, record):
        '''Area of polygon:Area of circle w/circumference = polygon's perimeter'''
        points = record['geometry']['coordinates'][0]
        sgeom = Polygon(points)
        ratio = 4 * pi * (sgeom.area/(sgeom.length**2))
        return ratio

    def calcSchwartzberg(self, record):
        '''Perimeter of polygon:Circumference of circle w/area = polygon's area'''
        points = record['geometry']['coordinates'][0]
        sgeom = Polygon(points)
        r = sqrt(sgeom.area/pi)
        cir = 2 * pi * r
        ratio = 1/(sgeom.length/cir)
        return ratio

    def calcConvexHull(self, record):
        '''Area of polygon:Area of minimum convex hull'''
        points = record['geometry']['coordinates'][0]
        sgeom = Polygon(points)
        hull = sgeom.convex_hull
        ratio = sgeom.area/hull.area
        return ratio

    def calcReock(self, record):
        '''Area of polygon:Area of minimum bounding circle'''
        points = record['geometry']['coordinates'][0]
        sgeom = Polygon(points)
        mb = miniball.Miniball(points)
        ca = pi * mb.squared_radius()
        ratio = sgeom.area/ca
        return ratio
        
testCompactness = CompactnessScores(inputData,outputData)

'''Folium map won't work when I try to add geojson layer. 
No errors, so was stuck with blind stabs. Best guess right now is
there are issues with the geojson (it opens fine in QGIS though)


#reproject to lat-lon for displaying & generate ranges for classifications
wgs84 = {'init':"EPSG:4326"}

map_layer = "final_out/districts_wgs1984.geojson"

with fiona.open(testCompactness.final_output) as in_polygons:
    reprojectPolygons(in_polygons, map_layer, "GeoJSON", in_polygons.schema.copy(), wgs84)
    
    #popo_ranges = getClassRanges(in_polygons, "Polsby-Popper", colors)
    #schwartz_ranges = getClassRanges(in_polygons, "Schwartzberg", colors)
    #ch_ranges = getClassRanges(in_polygons, "Convex-Hull", colors)
    #reock_ranges = getClassRanges(in_polygons, "Reock", colors)


# map
my_map = folium.Map(
            location=[41, -98],
            zoom_start=3,
            tiles='CartoDB Positron'
            )

popo = folium.GeoJson(map_layer,
                      name="Polsby-Popper",
                      style_function=lambda x: getColor(x, "Polsby-Popper", popo_ranges)
                         ).add_to(my_map)

schwartz = folium.GeoJson(map_layer,
                          name="Schwartzberg",
                          style_function=lambda x: getColor(x, "Schwartzberg", schwartz_ranges)
                         ).add_to(my_map)

ch = folium.GeoJson(map_layer,
                    name="Convex Hull",
                    style_function=lambda x: getColor(x, "Convex-Hull", ch_ranges)
                         ).add_to(my_map)

reock = folium.GeoJson(map_layer,
                       name="Reock",
                       style_function=lambda x: getColor(x, "Reock", reock_ranges)
                         ).add_to(my_map)


layer_control = folium.LayerControl()
my_map.add_child(layer_control)

stuff = folium.GeoJson(map_layer).add_to(my_map)

my_map
'''

North Carolina 2
Indiana 2
Virginia 2
districtShapes/test.shp Tranformation Complete
Schema Updated
Processing Successful


'Folium map won\'t work when I try to add geojson layer. \nNo errors, so was stuck with blind stabs. Best guess right now is\nthere are issues with the geojson (it opens fine in QGIS though)\n\n\n#reproject to lat-lon for displaying & generate ranges for classifications\nwgs84 = {\'init\':"EPSG:4326"}\n\nmap_layer = "final_out/districts_wgs1984.geojson"\n\nwith fiona.open(testCompactness.final_output) as in_polygons:\n    reprojectPolygons(in_polygons, map_layer, "GeoJSON", in_polygons.schema.copy(), wgs84)\n    \n    #popo_ranges = getClassRanges(in_polygons, "Polsby-Popper", colors)\n    #schwartz_ranges = getClassRanges(in_polygons, "Schwartzberg", colors)\n    #ch_ranges = getClassRanges(in_polygons, "Convex-Hull", colors)\n    #reock_ranges = getClassRanges(in_polygons, "Reock", colors)\n\n\n# map\nmy_map = folium.Map(\n            location=[41, -98],\n            zoom_start=3,\n            tiles=\'CartoDB Positron\'\n            )\n\npopo = folium.GeoJson(map_layer,\n            

In [ ]:
data_test = [
        {"type": "Feature",
         "id": 1,
        "properties": 
            {
                'myField':0.12
            }
        },
        {"type": "Feature",
         "id": 2,
        "properties": 
            {
                'myField':0.75
            }
        },
        {"type": "Feature",
         "id": 3,
        "properties": 
            {
                'myField':0.92
            }
        },
        {"type": "Feature",
         "id": 4,
        "properties": 
            {
                'myField':0.4
            }
        },
        {"type": "Feature",
         "id": 5,
        "properties": 
            {
                'myField':0.3
            }
        },
        {"type": "Feature",
         "id": 6,
        "properties": 
            {
                'myField':0.37
            }
        },
        {"type": "Feature",
         "id": 7,
        "properties": 
            {
                'myField':0.45
            }
        },
        {"type": "Feature",
         "id": 8,
        "properties": 
            {
                'myField':0.67
            }
        }
    ]

test = fiona.open("test_out/test_output.geojson")


colors = ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494']
colors.reverse()

def getClassRanges(inputData, field, colorList):
    '''construct list of ranges and corresponding color to use'''
    num_classes = len(colorList)
    colorList = colorList
    data = []
    classes = []
    ranges = []
    
    for feature in inputData:
        value = feature['properties'][field]
        if value != None:
            data.append(value)
        
    step = (max(data)-min(data))/num_classes
    
    for i in range(num_classes):
        classes.append(max(data)-(i * step))

    for i in range(len(classes)-1):
        ranges.append((classes[i], classes[i+1], colorList[i]))
    ranges.append((classes[-1], 0, colorList[-1]))
    return ranges

ranges = getClassRanges(test,'Polsby-Popper', colors)
print(ranges)

def getColor(feature, field, range_set):
    '''return fill color based on classification range'''
    val = feature['properties'][field]
    if val != None:
        for i in range(len(range_set)):
            upper = range_set[i][0]
            lower = range_set[i][1]
            color = range_set[i][2]
            if val <= upper and val > lower:
                return {'fillcolor':color}
                break
    else:
        return {'fillcolor':'#d3d3d3'}

for feature in test:
    print(getColor(feature,'Polsby-Popper', ranges))